In [1]:
from os import listdir
from pyspark import SQLContext
from pyspark.sql.functions import *

In [2]:
sqlContext = SQLContext(sc)

In [3]:
# with files on folder Dataset A insert on datasets/
# with page views json insert on datasets/page_views/

path_folder = 'datasets/'
table_names = ['subscriptions', 'sessions', 'students', 'subjects', 
               'universities', 'courses', 'student_follow_subject']

In [4]:
@udf
def quote_string_(data):
    if data:
        return data.split('@')[0]

In [5]:
#Process Page_views
file_name = 'page_views'
folder_files = listdir(path_folder+file_name)
page_views = sqlContext.read.json(path_folder+file_name+'/*.json')
split_col = split(page_views['studentId_clientType'], '@')
page_views = page_views.withColumn('page_views_studentsId', split_col.getItem(0))
page_views = page_views.withColumn('page_views_ClientType', split_col.getItem(1))
page_views = page_views.withColumn('at', concat(page_views.at.substr(0, 10), lit(' 00:00:00'))) 

In [6]:
for table in table_names:
    exec(f"{table} = sqlContext.read.json('{path_folder}{table}.json')")
    exec(f"columns_name = {table}.columns")
    for column in columns_name:
        exec(f"{table} = {table}.withColumnRenamed('{column}', '{table}_{column}')")
    print(f"PySpark Dataframe {table} created")

PySpark Dataframe subscriptions created
PySpark Dataframe sessions created
PySpark Dataframe students created
PySpark Dataframe subjects created
PySpark Dataframe universities created
PySpark Dataframe courses created
PySpark Dataframe student_follow_subject created


In [7]:
# Creating the 
tmp_df = students.join(courses, students["students_CourseId"] == courses["courses_Id"], "left_outer")
tmp_df = tmp_df.join(universities, tmp_df["students_UniversityId"] == universities["universities_Id"], "left_outer")
tmp_df = tmp_df.join(subscriptions, tmp_df["students_Id"] == subscriptions["subscriptions_StudentId"], "left_outer")
tmp_df = tmp_df.join(sessions, tmp_df["students_Id"] == sessions["sessions_StudentId"], "left_outer")
tmp_df = tmp_df.join(student_follow_subject, tmp_df["students_Id"] == student_follow_subject["student_follow_subject_StudentId"], "left_outer")
tmp_df = tmp_df.join(subjects, tmp_df["student_follow_subject_SubjectId"] == subjects["subjects_Id"], "left_outer")
tmp_df = tmp_df.join(page_views, tmp_df["students_Id"] == page_views["page_views_studentsId"], "inner")

In [8]:
tmp_df \
    .groupby('at', 'courses_name','universities_name', 'subscriptions_PlanType', 'students_city', 'students_StudentClient', 'subjects_name') \
    .count()

DataFrame[at: string, courses_name: string, universities_name: string, subscriptions_PlanType: string, students_city: string, students_StudentClient: string, subjects_name: string, count: bigint]

In [ ]:
## Writing table at bigquery, but its only possible to run in a dataproc cluster.
tmp_df.write.format('bigquery') \
  .option('table', 'Dataset.page_views_analytical') \
  .save()